In [5]:
import cv2
import numpy as np

def DIRECTOR(new_image, image_t):    
    image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY) 
    orb = cv2.ORB_create(100, 1.4)

    (kp1, des1) = orb.detectAndCompute(image_t, None)
    (kp2, des2) = orb.detectAndCompute(image, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    match = bf.match(des2,des1)
    match = sorted(match, key=lambda val: val.distance)    
    return len(match)

cam = cv2.VideoCapture(0)

image_2 = cv2.imread('images/right_1.png', 0)
image_1 = cv2.imread('images/left_1.png', 0) 
  

while True:
    ret, frame = cam.read()    
    h, w = frame.shape[:2]
    
    t_left_x = int(w/3)
    t_left_y = int((h/2) + (h/4))
    b_right_x = int((w/ 3)*2)
    b_right_y = int((h/2) - (h/4))
    
    b_right_x_1 = int((w/2))
    t_left_x_2 = int((w/2))

    cv2.rectangle(frame, (t_left_x,t_left_y), (b_right_x,b_right_y), 255, 2)
    cropped_1 = frame[b_right_y:t_left_y , t_left_x:b_right_x_1]
    
    cropped_2 = frame[b_right_y:t_left_y ,t_left_x_2:b_right_x]
    frame = cv2.flip(frame,1) 

#    cv2.imshow('Cropped1', cropped_1)
#    cv2.imshow('Cropped2', cropped_2)
    
    matches_1 = DIRECTOR(cropped_1, image_1)    
    matches_2 = DIRECTOR(cropped_2, image_2)
    
    output_string = "Right Matches = " + str(matches_2)
    cv2.putText(frame, output_string, (340,460), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
    
    output_string = "Left Matches = " + str(matches_1)
    cv2.putText(frame, output_string, (20,460), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
    
    threshold = 20    
   
    if matches_1 > threshold and matches_1 > matches_2:
            cv2.rectangle(frame, (t_left_x,t_left_y), (b_right_x,b_right_y), (0,225,255), 2)
            cv2.putText(frame,'GO LEFT',(170,45), cv2.FONT_HERSHEY_COMPLEX, 1.4 ,(0,0,255), 2)
    if matches_2 > threshold and matches_2 > matches_1:
            cv2.rectangle(frame, (t_left_x,t_left_y), (b_right_x,b_right_y), (0,225,255), 2)
            cv2.putText(frame,'GO RIGHT',(170,45), cv2.FONT_HERSHEY_COMPLEX, 1.4 ,(0,0,255), 2)
    
    cv2.imshow('Left', image_1)
    cv2.imshow('Right', image_2)
   
   
    cv2.imshow('GUIDE', frame)    
    if cv2.waitKey(1) == 13: 
        break

cam.release
cv2.destroyAllWindows()   